In [7]:
import os
import pandas as pd

# 指定本地数据集路径
path = r'C:\Users\fanh\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231'

csv_folder = os.path.join(path, "csv")

csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

csv_data = {file: pd.read_csv(os.path.join(csv_folder, file)) for file in csv_files}

for name, df in csv_data.items():
    print(f"{name} has {df.shape[1]} features:")
    print(df.columns.tolist())
    print("-" * 60)


common_player_info.csv has 33 features:
['person_id', 'first_name', 'last_name', 'display_first_last', 'display_last_comma_first', 'display_fi_last', 'player_slug', 'birthdate', 'school', 'country', 'last_affiliation', 'height', 'weight', 'season_exp', 'jersey', 'position', 'rosterstatus', 'games_played_current_season_flag', 'team_id', 'team_name', 'team_abbreviation', 'team_code', 'team_city', 'playercode', 'from_year', 'to_year', 'dleague_flag', 'nba_flag', 'games_played_flag', 'draft_year', 'draft_round', 'draft_number', 'greatest_75_flag']
------------------------------------------------------------
draft_combine_stats.csv has 47 features:
['season', 'player_id', 'first_name', 'last_name', 'player_name', 'position', 'height_wo_shoes', 'height_wo_shoes_ft_in', 'height_w_shoes', 'height_w_shoes_ft_in', 'weight', 'wingspan', 'wingspan_ft_in', 'standing_reach', 'standing_reach_ft_in', 'body_fat_pct', 'hand_length', 'hand_width', 'standing_vertical_leap', 'max_vertical_leap', 'lane_agil

In [42]:
# 提取 game.csv 数据
game_df = csv_data['game.csv']

if 'pts_home' in game_df.columns and 'pts_away' in game_df.columns:
    game_df['result'] = (game_df['pts_home'] > game_df['pts_away']).astype(int)
    print(game_df[['pts_home', 'pts_away', 'result']].head())
else:
    print("The required columns 'pts_home' and 'pts_away' are not present in the dataset.")

game_df['game_date'] = pd.to_datetime(game_df['game_date'])
game_df_1999_onwards = game_df[game_df['game_date'].dt.year >= 1999]
print(game_df_1999_onwards[['game_date', 'pts_home', 'pts_away', 'result']].head())


   pts_home  pts_away  result
0      66.0      68.0       0
1      56.0      51.0       1
2      59.0      53.0       1
3      63.0      47.0       1
4      33.0      50.0       0
       game_date  pts_home  pts_away  result
34059 1999-02-05     101.0      83.0       1
34060 1999-02-05      92.0     101.0       0
34061 1999-02-05     104.0      96.0       1
34062 1999-02-05      96.0      81.0       1
34063 1999-02-05      66.0      78.0       0


In [43]:
game_df_1999_onwards = game_df_1999_onwards.dropna(subset=['result'])

y = game_df_1999_onwards['result']

y.head()


34059    1
34060    0
34061    1
34062    1
34063    0
Name: result, dtype: int64

In [44]:
# Extract play_by_play.csv data
play_by_play_df = csv_data['play_by_play.csv']

# Display the first few rows
play_by_play_df.head()



,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,29600012,0,12,0,1,14:43 PM,12:00,NaN,Start of 1st Period (14:43 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,29600012,2,10,0,1,14:50 PM,12:00,Jump Ball O'Neal vs. Kleine: Tip to Cassell,NaN,NaN,...,Suns,PHX,5.0,208,Sam Cassell,1.610613e+09,Phoenix,Suns,PHX,0
2,29600012,3,2,1,1,14:51 PM,11:45,NaN,NaN,MISS Cassell 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
3,29600012,4,4,0,1,14:51 PM,11:43,O'Neal REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,29600012,5,2,1,1,14:51 PM,11:29,MISS Ceballos 26' 3PT Jump Shot,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


In [45]:
# 统计胜负情况
new_table = game_df_1999_onwards[['game_id', 'game_date', 'team_abbreviation_home', 'team_abbreviation_away', 'result']]

# 重命名列名
new_table.columns = ['Game ID', 'Game Date', 'Home Team', 'Away Team', 'Home Win']

# 显示前几行
print(new_table.head())

        Game ID  Game Date Home Team Away Team  Home Win
34059  29800007 1999-02-05       SAS       SAC         1
34060  29800010 1999-02-05       LAC       PHX         0
34061  29800009 1999-02-05       UTA       CHI         1
34062  29800006 1999-02-05       IND       WAS         1
34063  29800004 1999-02-05       CHH       PHI         0


In [46]:
from datetime import timedelta

# 定义一个函数计算最近N场比赛的胜率
def calculate_recent_win_rate(df, team_col, result_col, date_col, team, current_date, n):
    recent_games = df[(df[team_col] == team) & (df[date_col] < current_date)].sort_values(by=date_col, ascending=False).head(n)
    if recent_games.empty:
        return 0
    return recent_games[result_col].mean()

# 定义一个函数计算最近N个月的胜率
def calculate_recent_months_win_rate(df, team_col, result_col, date_col, team, current_date, months):
    start_date = current_date - timedelta(days=30 * months)
    recent_games = df[(df[team_col] == team) & (df[date_col] < current_date) & (df[date_col] >= start_date)]
    if recent_games.empty:
        return 0
    return recent_games[result_col].mean()


In [47]:

# 创建新表格
new_table_with_win_rates = []

for _, row in game_df_1999_onwards.iterrows():
    game_id = row['game_id']
    game_date = row['game_date']
    home_team = row['team_abbreviation_home']
    away_team = row['team_abbreviation_away']
    
    # 计算主场和客场球队的最近10场胜率
    home_recent_10_win_rate = calculate_recent_win_rate(game_df_1999_onwards, 'team_abbreviation_home', 'result', 'game_date', home_team, game_date, 10)
    away_recent_10_win_rate = calculate_recent_win_rate(game_df_1999_onwards, 'team_abbreviation_away', 'result', 'game_date', away_team, game_date, 10)
    
    # 计算主场和客场球队的最近3个月胜率
    home_recent_3_months_win_rate = calculate_recent_months_win_rate(game_df_1999_onwards, 'team_abbreviation_home', 'result', 'game_date', home_team, game_date, 6)
    away_recent_3_months_win_rate = calculate_recent_months_win_rate(game_df_1999_onwards, 'team_abbreviation_away', 'result', 'game_date', away_team, game_date, 6)
    
    # 添加到新表格
    new_table_with_win_rates.append({
        'Game ID': game_id,
        'Game Date': game_date,
        'Home Team Recent 10 Win Rate': home_recent_10_win_rate,
        'Away Team Recent 10 Win Rate': away_recent_10_win_rate,
        'Home Team Recent 3 Months Win Rate': home_recent_3_months_win_rate,
        'Away Team Recent 3 Months Win Rate': away_recent_3_months_win_rate
    })

# 转换为DataFrame
new_table_with_win_rates_df = pd.DataFrame(new_table_with_win_rates)

# 显示前几行
new_table_with_win_rates_df.head(30)

,Game ID,Game Date,Home Team Recent 10 Win Rate,Away Team Recent 10 Win Rate,Home Team Recent 3 Months Win Rate,Away Team Recent 3 Months Win Rate
0,29800007,1999-02-05,0.0,0.0,0.0,0.0
1,29800010,1999-02-05,0.0,0.0,0.0,0.0
2,29800009,1999-02-05,0.0,0.0,0.0,0.0
3,29800006,1999-02-05,0.0,0.0,0.0,0.0
4,29800004,1999-02-05,0.0,0.0,0.0,0.0
5,29800011,1999-02-05,0.0,0.0,0.0,0.0
6,29800005,1999-02-05,0.0,0.0,0.0,0.0
7,29800002,1999-02-05,0.0,0.0,0.0,0.0
8,29800012,1999-02-05,0.0,0.0,0.0,0.0
9,29800008,1999-02-05,0.0,0.0,0.0,0.0


In [49]:
# 筛选2000年之后的数据
filtered_data = new_table_with_win_rates_df[new_table_with_win_rates_df['Game Date'].dt.year >= 2000]

# 保存到CSV文件
filtered_data.to_csv('new_table_with_win_rates_2000_onwards.csv', index=False)

In [51]:
def days_since_last_game(df, team_col, date_col, team, current_date):
    """
    计算距离上一次比赛的天数
    """
    previous_games = df[(df[team_col] == team) & (df[date_col] < current_date)].sort_values(by=date_col, ascending=False)
    if previous_games.empty:
        return None  # 如果没有之前的比赛，返回 None
    last_game_date = previous_games.iloc[0][date_col]
    return (current_date - last_game_date).days


def number_of_games_in_last_7_days(df, team_col, date_col, team, current_date):
    """
    计算最近一周的比赛场次
    """
    start_date = current_date - pd.Timedelta(days=7)
    recent_games = df[(df[team_col] == team) & (df[date_col] < current_date) & (df[date_col] >= start_date)]
    return len(recent_games)


def is_back_to_back(df, team_col, date_col, team, current_date):
    """
    判断是否为背靠背比赛
    """
    days_since_last = days_since_last_game(df, team_col, date_col, team, current_date)
    return days_since_last == 1  # 如果距离上一次比赛为1天，则为背靠背比赛

In [56]:
# 创建新表格
new_table_with_game_stats = []

for _, row in game_df_1999_onwards.iterrows():
    game_id = row['game_id']
    game_date = row['game_date']
    home_team = row['team_abbreviation_home']
    away_team = row['team_abbreviation_away']
    
    # 计算主队的统计数据
    home_days_since_last_game = days_since_last_game(game_df_1999_onwards, 'team_abbreviation_home', 'game_date', home_team, game_date)
    home_games_last_7_days = number_of_games_in_last_7_days(game_df_1999_onwards, 'team_abbreviation_home', 'game_date', home_team, game_date)
    home_is_back_to_back = is_back_to_back(game_df_1999_onwards, 'team_abbreviation_home', 'game_date', home_team, game_date)
    
    # 计算客队的统计数据
    away_days_since_last_game = days_since_last_game(game_df_1999_onwards, 'team_abbreviation_away', 'game_date', away_team, game_date)
    away_games_last_7_days = number_of_games_in_last_7_days(game_df_1999_onwards, 'team_abbreviation_away', 'game_date', away_team, game_date)
    away_is_back_to_back = is_back_to_back(game_df_1999_onwards, 'team_abbreviation_away', 'game_date', away_team, game_date)
    
    # 添加到新表格
    new_table_with_game_stats.append({
        'Game ID': game_id,
        'Game Date': game_date,
        'Home Days Since Last Game': home_days_since_last_game,
        'Home Games Last 7 Days': home_games_last_7_days,
        'Home Is Back-to-Back': home_is_back_to_back,
        'Away Days Since Last Game': away_days_since_last_game,
        'Away Games Last 7 Days': away_games_last_7_days,
        'Away Is Back-to-Back': away_is_back_to_back
    })

# 转换为DataFrame
new_table_with_game_stats_df = pd.DataFrame(new_table_with_game_stats)

# 显示前几行
new_table_with_game_stats_df.head()

,Game ID,Game Date,Home Days Since Last Game,Home Games Last 7 Days,Home Is Back-to-Back,Away Days Since Last Game,Away Games Last 7 Days,Away Is Back-to-Back
0,29800007,1999-02-05,NaN,0,False,NaN,0,False
1,29800010,1999-02-05,NaN,0,False,NaN,0,False
2,29800009,1999-02-05,NaN,0,False,NaN,0,False
3,29800006,1999-02-05,NaN,0,False,NaN,0,False
4,29800004,1999-02-05,NaN,0,False,NaN,0,False


In [59]:
# 筛选2000年之后的数据
filtered_data_2000_onwards = new_table_with_game_stats_df[new_table_with_game_stats_df['Game Date'].dt.year >= 2000]

# 保存到CSV文件
filtered_data_2000_onwards.to_csv('game_stats_2000_onwards.csv', index=False)